# DATASETS

In [7]:
def unsorted_segment_sum(data, segment_ids, num_segments):
    """Custom PyTorch op to replicate TensorFlow's `unsorted_segment_sum`."""
    result_shape = (num_segments, data.size(1))
    result = data.new_full(result_shape, 0)  # Init empty result tensor.
    segment_ids = segment_ids.unsqueeze(-1).expand(-1, data.size(1))
    result.scatter_add_(0, segment_ids, data)
    return result

In [8]:
import torch
# from n_body_system.dataset_nbody import NBodyDataset
from dataset import NBodyDataset
# from n_body_system.model import EGNN_vel
from EGNN import EGNN_vel
# from functions_utils import unsorted_segment_mean
import os
from torch import nn, optim
import json

In [9]:
model_name = 'egnn_task2'
batch_size = 100
epochs = 1000
no_cuda = False
seed = 1
log_interval = 1
test_interval = 5
outf = './logs'
lr = 5e-4
nf = 64
attention = 0
n_layers = 4
degree = 2
max_training_samples = 3000
dataset = "nbody_small"
sweep_training = 0
time_exp = 0
weight_decay = 1e-12
div = 1
norm_diff = False
tanh = False

time_exp_dic = {'time': 0, 'counter': 0}

cuda = not no_cuda and torch.cuda.is_available()

In [10]:
# Create log directory
try:
    os.makedirs(outf)
except OSError:
    pass

try:
    os.makedirs(outf + "/" + model_name)
except OSError:
    pass

In [11]:
device = torch.device("cuda" if cuda else "cpu")
loss_mse = nn.MSELoss()

In [12]:
def get_velocity_attr(loc, vel, rows, cols):

    diff = loc[cols] - loc[rows]
    norm = torch.norm(diff, p=2, dim=1).unsqueeze(1)
    u = diff/norm
    va, vb = vel[rows] * u, vel[cols] * u
    va, vb = torch.sum(va, dim=1).unsqueeze(1), torch.sum(vb, dim=1).unsqueeze(1)
    return va

In [13]:
# # Datasets and loaders
# dataset_train = NBodyDataset(partition='train', dataset_name=dataset,
#                                  max_samples=max_training_samples)
# loader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)

# dataset_val = NBodyDataset(partition='val', dataset_name="nbody_small")
# loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=batch_size, shuffle=False, drop_last=False)

# dataset_test = NBodyDataset(partition='test', dataset_name="nbody_small")
# loader_test = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=False, drop_last=False)

In [14]:
import glob
from torch_geometric.loader import DataLoader

paths = glob.glob("data/task1_2/train/*.npz")

train_paths = paths[:int(len(paths)*0.8)]
val_paths = paths[int(len(paths)*0.8):]

dataset_train = NBodyDataset(train_paths)
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=False, drop_last=True) # TODO: shuffle=True

dataset_val = NBodyDataset(val_paths)
loader_val = DataLoader(dataset_val, batch_size=batch_size, shuffle=False)

dataset_test = NBodyDataset(val_paths)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)



In [15]:
dataset_train[0]

Data(x=[3, 1], edge_index=[2, 6], edge_attr=[6, 2], loc=[3, 2], vel=[3, 2], next_loc=[3, 2], next_vel=[3, 2])

In [16]:
print(type(loader_train))
next(iter(loader_train))


<class 'torch_geometric.loader.dataloader.DataLoader'>


DataBatch(x=[24, 1], edge_index=[2, 48], edge_attr=[48, 2], loc=[24, 2], vel=[24, 2], next_loc=[24, 2], next_vel=[24, 2], batch=[24], ptr=[9])

In [17]:
# # Location at time 0, velocity at time 0, edge_attr, charges, location at time T
# dataset_train[0][0].shape, dataset_train[0][1].shape, dataset_train[0][2].shape, dataset_train[0][3].shape, dataset_train[0][4].shape

In [18]:
# print(dataset_train.edges)

In [19]:
# print(dataset_train.get_edges(1, 3))

# MODEL

In [24]:
from torch_geometric.nn import summary

model = EGNN_vel(in_node_nf=1, in_edge_nf=2, hidden_nf=nf, device=device, n_layers=n_layers, recurrent=True, norm_diff=norm_diff, tanh=tanh).to(device)

dummy_data = dataset_train[0]
print(summary(model, dummy_data.x, dummy_data.loc, dummy_data.edge_index, dummy_data.vel, dummy_data.edge_attr))


+----------------------------------+----------------------------------------+-------------------------+----------+
| Layer                            | Input Shape                            | Output Shape            | #Param   |
|----------------------------------+----------------------------------------+-------------------------+----------|
| EGNN_vel                         | [3, 1], [3, 2], [2, 6], [3, 2], [6, 2] | [3, 2], [3, 2]          | 100,677  |
| ├─(embedding)Linear              | [3, 1]                                 | [3, 64]                 | 128      |
| ├─(gcl_0)E_GCL_vel               | [3, 64], [2, 6], [3, 2], [3, 2]        | [3, 64], [3, 2], [6, 2] | 33,473   |
| │    └─(edge_mlp)Sequential      | [6, 131]                               | [6, 64]                 | 12,608   |
| │    │    └─(0)Linear            | [6, 131]                               | [6, 64]                 | 8,448    |
| │    │    └─(1)SiLU              | [6, 64]                                | [6

# TRAINING

In [25]:
optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [26]:
def training_procedure(model, optimizer, loader_train, loader_val, loader_test, epochs, test_interval, model_name, outf):
    results = {'epochs': [], 'losess': []}
    best_val_loss = 1e8
    best_test_loss = 1e8
    best_epoch = 0
    for epoch in range(0, epochs):
        train(model, optimizer, epoch, loader_train)
        if epoch % test_interval == 0:
            #train(epoch, loader_train, backprop=False)
            val_loss = train(model, optimizer, epoch, loader_val, backprop=False)
            test_loss = train(model, optimizer, epoch, loader_test, backprop=False)
            results['epochs'].append(epoch)
            results['losess'].append(test_loss)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_test_loss = test_loss
                best_epoch = epoch
                torch.save(model.state_dict(), outf + "/" + model_name + f"/{model_name}.pth")
            print("*** Best Val Loss: %.5f \t Best Test Loss: %.5f \t Best epoch %d" % (best_val_loss, best_test_loss, best_epoch))

        json_object = json.dumps(results, indent=4)
        with open(outf + "/" + model_name + "/losess.json", "w") as outfile:
            outfile.write(json_object)
    return best_val_loss, best_test_loss, best_epoch
    

In [27]:
def train(model, optimizer, epoch, loader, backprop=True):
    if backprop:
        model.train()
    else:
        model.eval()

    res = {'epoch': epoch, 'loss': 0, 'coord_reg': 0, 'counter': 0}

    # for batch_idx, data in enumerate(loader):
    for data in loader:
        # print(f"Data type: {type(data)}")
        # print(f"Data: {data}")
        # # print("\nBatch idx: ", batch_idx)
        # print("Data: ", len(data))
        # batch_size, n_nodes, _ = data[0].size()
        # print("Batch size: ", batch_size)
        # print("Number of nodes: ", n_nodes)
        
        # data = [d.to(device) for d in data]

        # print("Data: ", data[0].shape)
        # data = [d.view(-1, d.size(2)) for d in data] # Remove the batch dimension
        # print("Data: ", data[0].shape)

        # loc, vel, edge_attr, charges, loc_end = data
        loc = data.loc
        vel = data.vel
        x = data.x
        edges = data.edge_index
        edge_attr = data.edge_attr
        loc_end = data.next_loc
        vel_end = data.next_vel

        # loc, vel, x, edges, edge_attr, loc_end = data # TODO posso farlgli passare anche gli edges

        # edges = loader.dataset.get_edges(batch_size, n_nodes) # TODO questa va tolta
        edges = [edges[0].to(device), edges[1].to(device)]

        """
            Batch idx:  0
            Data: ...
            Batch size:  100
            Number of nodes:  5
            Data:  torch.Size([100, 5, 3])
            Data:  torch.Size([500, 3])
            train epoch 18 avg loss: 0.01948
        """

        optimizer.zero_grad()

        # if time_exp:
        #     torch.cuda.synchronize()
        #     t1 = time.time()
        

        # nodes = torch.sqrt(torch.sum(vel ** 2, dim=1)).unsqueeze(1).detach()
        # rows, cols = edges
        # loc_dist = torch.sum((loc[rows] - loc[cols])**2, 1).unsqueeze(1)  # relative distances among locations
        # edge_attr = torch.cat([edge_attr, loc_dist], 1).detach()  # concatenate all edge properties

        # loc_pred = model(nodes, loc.detach(), edges, vel, edge_attr)
        # loc_pred = model(x, loc.detach(), edges, vel, edge_attr)
        loc_pred, vel_pred = model(x, loc.detach(), edges, vel, edge_attr)


        # if time_exp:
        #     torch.cuda.synchronize()
        #     t2 = time.time()
        #     time_exp_dic['time'] += t2 - t1
        #     time_exp_dic['counter'] += 1

            # print("Forward average time: %.6f" % (time_exp_dic['time'] / time_exp_dic['counter']))
        
        # loss = loss_mse(loc_pred, loc_end)
        loss = loss_mse(vel_pred, vel_end)

        if backprop:
            loss.backward()
            optimizer.step()

        res['loss'] += loss.item()*batch_size
        res['counter'] += batch_size

        # if batch_idx % log_interval == 0 and (model == "se3_transformer" or model == "tfn"):
        #     print('===> {} Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(loader.dataset.partition,
        #         epoch, batch_idx * batch_size, len(loader.dataset),
        #         100. * batch_idx / len(loader),
        #         loss.item()))

    if not backprop:
        prefix = "==> "
    else:
        prefix = ""
    print('%s epoch %d avg loss: %.5f' % (prefix, epoch, res['loss'] / res['counter']))
    # print('%s epoch %d avg loss: %.5f' % (prefix+loader.dataset.partition, epoch, res['loss'] / res['counter']))

    return res['loss'] / res['counter']


In [28]:
# training_procedure(model, optimizer, loader_train, loader_val, loader_test, epochs, test_interval, model_name, outf)

# Other Tests

In [29]:
# Smaller batch size
model_name = 'egnn_task2_2'
batch_size = 4
epochs = 300
no_cuda = False
seed = 1
log_interval = 1
test_interval = 5
outf = './logs'
lr = 5e-4
nf = 64
attention = 0
n_layers = 4
degree = 2
max_training_samples = 3000
dataset = "nbody_small"
sweep_training = 0
time_exp = 0
weight_decay = 1e-12
div = 1
norm_diff = False
tanh = False

time_exp_dic = {'time': 0, 'counter': 0}

# Create log directory
try:
    os.makedirs(outf)
except OSError:
    pass

try:
    os.makedirs(outf + "/" + model_name)
except OSError:
    pass

cuda = not no_cuda and torch.cuda.is_available()

# __________________________________________________________________________________________________ #

model = EGNN_vel(in_node_nf=1, in_edge_nf=2, hidden_nf=nf, device=device, n_layers=n_layers, recurrent=True, norm_diff=norm_diff, tanh=tanh).to(device)

dummy_data = dataset_train[0]
print(summary(model, dummy_data.x, dummy_data.loc, dummy_data.edge_index, dummy_data.vel, dummy_data.edge_attr))

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
training_procedure(model, optimizer, loader_train, loader_val, loader_test, epochs, test_interval, model_name, outf)

+----------------------------------+----------------------------------------+-------------------------+----------+
| Layer                            | Input Shape                            | Output Shape            | #Param   |
|----------------------------------+----------------------------------------+-------------------------+----------|
| EGNN_vel                         | [3, 1], [3, 2], [2, 6], [3, 2], [6, 2] | [3, 2], [3, 2]          | 134,150  |
| ├─(embedding)Linear              | [3, 1]                                 | [3, 64]                 | 128      |
| ├─(gcl_0)E_GCL_vel               | [3, 64], [2, 6], [3, 2], [3, 2]        | [3, 64], [3, 2], [6, 2] | 33,473   |
| │    └─(edge_mlp)Sequential      | [6, 131]                               | [6, 64]                 | 12,608   |
| │    │    └─(0)Linear            | [6, 131]                               | [6, 64]                 | 8,448    |
| │    │    └─(1)SiLU              | [6, 64]                                | [6

KeyboardInterrupt: 

In [30]:
# Smaller model
model_name = 'egnn_task2_3'
batch_size = 16
epochs = 300
no_cuda = False
seed = 1
log_interval = 1
test_interval = 5
outf = './logs'
lr = 5e-4
nf = 32
attention = 0
n_layers = 4
degree = 2
max_training_samples = 3000
dataset = "nbody_small"
sweep_training = 0
time_exp = 0
weight_decay = 1e-12
div = 1
norm_diff = False
tanh = False

time_exp_dic = {'time': 0, 'counter': 0}

# Create log directory
try:
    os.makedirs(outf)
except OSError:
    pass

try:
    os.makedirs(outf + "/" + model_name)
except OSError:
    pass

cuda = not no_cuda and torch.cuda.is_available()

# __________________________________________________________________________________________________ #

model = EGNN_vel(in_node_nf=1, in_edge_nf=2, hidden_nf=nf, device=device, n_layers=n_layers, recurrent=True, norm_diff=norm_diff, tanh=tanh).to(device)

dummy_data = dataset_train[0]
print(summary(model, dummy_data.x, dummy_data.loc, dummy_data.edge_index, dummy_data.vel, dummy_data.edge_attr))

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
training_procedure(model, optimizer, loader_train, loader_val, loader_test, epochs, test_interval, model_name, outf)

+----------------------------------+----------------------------------------+-------------------------+----------+
| Layer                            | Input Shape                            | Output Shape            | #Param   |
|----------------------------------+----------------------------------------+-------------------------+----------|
| EGNN_vel                         | [3, 1], [3, 2], [2, 6], [3, 2], [6, 2] | [3, 2], [3, 2]          | 34,310   |
| ├─(embedding)Linear              | [3, 1]                                 | [3, 32]                 | 64       |
| ├─(gcl_0)E_GCL_vel               | [3, 32], [2, 6], [3, 2], [3, 2]        | [3, 32], [3, 2], [6, 2] | 8,545    |
| │    └─(edge_mlp)Sequential      | [6, 67]                                | [6, 32]                 | 3,232    |
| │    │    └─(0)Linear            | [6, 67]                                | [6, 32]                 | 2,176    |
| │    │    └─(1)SiLU              | [6, 32]                                | [6

In [ ]:
# Lowe lr
model_name = 'egnn_task2_4'
batch_size = 16
epochs = 300
no_cuda = False
seed = 1
log_interval = 1
test_interval = 5
outf = './logs'
lr = 1e-4
nf = 64
attention = 0
n_layers = 4
degree = 2
max_training_samples = 3000
dataset = "nbody_small"
sweep_training = 0
time_exp = 0
weight_decay = 1e-12
div = 1
norm_diff = False
tanh = False

time_exp_dic = {'time': 0, 'counter': 0}

cuda = not no_cuda and torch.cuda.is_available()

# Create log directory
try:
    os.makedirs(outf)
except OSError:
    pass

try:
    os.makedirs(outf + "/" + model_name)
except OSError:
    pass

# __________________________________________________________________________________________________ #

model = EGNN_vel(in_node_nf=1, in_edge_nf=2, hidden_nf=nf, device=device, n_layers=n_layers, recurrent=True, norm_diff=norm_diff, tanh=tanh).to(device)

dummy_data = dataset_train[0]
print(summary(model, dummy_data.x, dummy_data.loc, dummy_data.edge_index, dummy_data.vel, dummy_data.edge_attr))

optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
training_procedure(model, optimizer, loader_train, loader_val, loader_test, epochs, test_interval, model_name, outf)